# BITE Data Generation

This notebook is intended to be used in conjunction with an E2E notebook (hereinafter referred to as the main notebook).

### 1.1 Prerequisites

Before running this notebook, ensure the following is met: 
* A namespace running from the [SKA-mid-psi](https://gitlab.com/ska-telescope/ska-mid-psi) pipeline deploy step, or a compatible one.
* A virtual env with [poetry](https://python-poetry.org/docs/basic-usage/#installing-dependencies) run on it to ensure all requirements are installed.
* Python 3.10 and the above venv selected as the interpreter for the notebook. 

### 1.2 Set Variables

Update the namespace `ns` and the `test_id` if necessary and set the `TANGO_HOST` environment variable. 

In [ ]:
import json
import os
from tango import Database, DevFailed, DeviceProxy

# --Required Vars--
ns = "ci-ska-mid-psi-1387054764-maecst"  # UPDATE THIS FOR RUN

test_id = "talon3 basic gaussian noise"  # UPDATE THIS if a diff dataset from the CBF Input Data is needed to generate the BITE data later

TANGO_HOST = "databaseds-tango-base." + ns + ".svc.cluster.local:10000"
print("TANGO DB HOST: ", TANGO_HOST)
os.environ["TANGO_HOST"] = TANGO_HOST

### 1.3 Set Config Variables

Set the locations of JSON files for configuring BITE generation. These should not need to be changed.

In [ ]:
print("Getting files...")

# Parent directory to use to grab config files.
DATA_DIR = os.path.join(os.path.dirname(os.path.dirname(os.getcwd())), "data")

# Config file directories
CBF_CONFIGS = os.path.join(DATA_DIR, "psi/cbf")
CBF_INPUT_DIR = os.path.join(CBF_CONFIGS, "cbf_input_data")

# Config files needed for BITE generation
CBF_INPUT_FILE = f"{CBF_INPUT_DIR}/cbf_input_data.json"
BITE_CONFIG_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/bite_configs.json"
FILTERS_FILE = f"{CBF_INPUT_DIR}/bite_config_parameters/filters.json"

files = [
    CBF_INPUT_FILE,
    BITE_CONFIG_FILE,
    FILTERS_FILE,
]

for file in files:
    if os.path.isfile(file):
        print(f"{file} exists: ✔️")
    else:
        print(f"{file} does not exist ❌")

print("\nDONE")

## 2 Load in Config Data

NOTE: This step needs to execute after the Load Dish Config Command has been executed in the main notebook.

Create a device proxy to the BITE device server and load in the necessary configuration files.

In [ ]:
bite = DeviceProxy("mid_csp_cbf/ec/bite")
tmc_csp_master = DeviceProxy("ska_mid/tm_leaf_node/csp_master")

dishVccConfig = json.loads(tmc_csp_master.dishVccConfig)
print(f"dishVccConfig from CSP Master: \n{dishVccConfig}\n")

with open(CBF_INPUT_FILE, encoding="utf-8") as f:
    cbf_input_json = json.load(f)["cbf_input_data"][test_id]
    receptors = cbf_input_json["receptors"]

    # ensure that the k-values in the cbf_input_json correctly match
    # the k-values from the dishVccConfig that was loaded in earlier
    for receptor in receptors:
        dish_id = receptor["dish_id"]
        k_value = dishVccConfig["dish_parameters"][dish_id]["k"]
        receptor["sample_rate_k"]=k_value

    cbf_input_data = json.dumps(cbf_input_json)
    print("CBF Input Data used to generate BITE data:")
    print(cbf_input_data)

bite.load_cbf_input_data(cbf_input_data)

In [ ]:
with open(BITE_CONFIG_FILE, encoding="utf-8") as f:
    bite_config_data = json.dumps(json.load(f))
    print("BITE configs:\n")
    print(bite_config_data)

bite.load_bite_config_data(bite_config_data)

In [ ]:
with open(FILTERS_FILE, encoding="utf-8") as f:
    filter_data = json.dumps(json.load(f))
    print("Filters:\n")
    print(filter_data)

bite.load_filter_data(filter_data)

## 3 Generate the BITE data

After the TelescopeOn command is executed in the main notebook, the generate_bite_data command can be executed:

In [ ]:
# monitor the kubectl logs
# kubectl logs -n $ns ds-bite-bite-0 -f

bite.set_timeout_millis(240000)
bite.generate_bite_data()
bite.set_timeout_millis(3000)

PAUSE: Return to the main notebook to execute Assign Resources.

## 4 Start LSTV Replay

When Assign Resources is completed, return to this notebook to start LSTV replay:

In [ ]:
print("Using the bite device to start lstv replay...")
bite.start_lstv_replay()

PAUSE: Return to the main notebook to execute Configure Scan and Scan.

## 5 Stop LSTV Replay

Once Scan and End Scan are executed, return to this notebook to stop LSTV replay:

In [ ]:
bite.stop_lstv_replay()

END: Return to the main notebook to finish the clean up steps: Release Resources, Telescope Off.